In [ ]:
!pip install groq==0.9.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 4.8 MB/s eta 0:00:00


In [ ]:
pip install youtube-transcript-api requests typing


In [ ]:
import re
from youtube_transcript_api import YouTubeTranscriptApi
import requests
from typing import List, Optional
import logging

In [ ]:

class TranscriptError(Exception):
    """Custom exception for transcript-related errors."""
    pass

def fetch_transcript(video_url: str) -> str:

    try:
        
        video_id_match = re.search(
            r"(?:v=|\/|youtu\.be\/|embed\/)([a-zA-Z0-9_-]{11})",
            video_url
        )
        if not video_id_match:
            raise TranscriptError("Invalid YouTube URL format")

        video_id = video_id_match.group(1)

        try:
            transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
            # Try to get English transcript first, fall back to auto-generated
            try:
                transcript = transcript_list.find_transcript(['en'])
            except:
                transcript = transcript_list.find_generated_transcript(['en'])

            transcript_data = transcript.fetch()
            return " ".join(item['text'] for item in transcript_data)

        except Exception as e:
            available_transcripts = []
            try:
                available_transcripts = [
                    t.language_code
                    for t in transcript_list.manual_transcripts
                ]
            except:
                pass

            raise TranscriptError(
                f"Failed to fetch transcript. Available languages: {available_transcripts}. Error: {str(e)}"
            )

    except Exception as e:
        raise TranscriptError(f"Error processing video: {str(e)}")

def preprocess_transcript(transcript: str, chunk_size: int = 200) -> List[str]:

    # Split on sentence boundaries instead of spaces
    sentences = [s.strip() for s in transcript.split('.') if s.strip()]
    chunks = []
    current_chunk = []
    current_length = 0

    for sentence in sentences:
        sentence_length = len(sentence)
        if current_length + sentence_length <= chunk_size:
            current_chunk.append(sentence)
            current_length += sentence_length
        else:
            if current_chunk:
                chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentence]
            current_length = sentence_length

    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')

    return chunks

def query_llama_model(api_key: str, prompt: str) -> Optional[str]:

    if not api_key:
        raise ValueError("API key is required")

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    payload = {
        "model": "llama-3.1-8b-instant",  
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant that answers questions about video content based on transcripts."
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.7

    }

    try:
        response = requests.post(
            "https://api.groq.com/openai/v1/chat/completions",
            headers=headers,
            json=payload,
            timeout=30
        )

        if response.status_code != 200:
            logging.error(f"API Error: {response.status_code} - {response.text}")
            return None

        result = response.json()
        if not result.get("choices"):
            raise ValueError("No choices in response")

        return result["choices"][0]["message"]["content"].strip()

    except requests.exceptions.RequestException as e:
        logging.error(f"API request failed: {str(e)}")
        return None
    except (KeyError, ValueError) as e:
        logging.error(f"Error parsing API response: {str(e)}")
        return None

def youtube_qa_system(video_url: str, question: str, api_key: str) -> str:

    try:
        
        if not video_url or not question or not api_key:
            raise ValueError("Missing required parameters")

        
        transcript = fetch_transcript(video_url)
        if not transcript:
            return "Could not extract transcript from video"

        
        chunks = preprocess_transcript(transcript)
        if not chunks:
            return "Failed to process transcript"


        context = " ".join(chunks)
        prompt = (
            f"Based on the following video transcript, please answer the question.\n\n"
            f"Transcript: {context}\n\n"
            f"Question: {question}\n\n"
            f"Answer:"
        )

        
        answer = query_llama_model(api_key, prompt)
        if not answer:
            return "Failed to generate answer"

        return answer

    except Exception as e:
        logging.error(f"Error in QA system: {str(e)}")
        return f"An error occurred: {str(e)}"


if __name__ == "__main__":
    video_url = "https://youtu.be/tbnzAVRZ9Xc?feature=shared"
    question = input("Enter Question ")
    groq_api_key = ""  # Replace with your Groq API key for Llama.

    logging.basicConfig(level=logging.INFO)
    answer = youtube_qa_system(video_url, question, groq_api_key)
    print("Answer:", answer)

Enter Question what is this video about?
Answer: This video is about the importance of taking risks and embracing failure in life. The speaker emphasizes that taking risks and stepping out of one's comfort zone is necessary for personal growth, success, and making a positive impact on the world. They encourage the graduates to be bold, pursue their passions, and not be afraid to fail, using their own experiences and examples to illustrate their points. The video also touches on the idea that failure can sometimes lead to unexpected opportunities and that it's essential to stay open-minded, learn from one's mistakes, and keep moving forward. The speaker's message is meant to inspire and motivate the graduates to make the most of their lives and talents.
